In [0]:
# import the necessary packages

## this is warnings module which is use in ignoring the warnings of the program #
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')


## Data manupulations
import scipy, scipy.ndimage
from skimage.util import random_noise
from PIL import Image
import numpy as np
from tqdm import tnrange, tqdm_notebook

In [0]:
def NearestNeighbors(matrix, indices):
    """
        k-NN: This is function will return the neighbors associated to given indices in the
              given matrix.
        Parameter:
        ----------
            Matrix : N-dimentional matrix.
            indices : The coordinates of the given Matrix
    
    """
    matrix = np.array(matrix)
    indices = tuple(np.transpose(np.atleast_2d(indices)))

    array_shape = matrix.shape

    dist = np.ones(array_shape)
    dist[indices] = 0

    dist = scipy.ndimage.distance_transform_cdt(dist, metric='chessboard')
    nb_indices = np.transpose(np.nonzero(dist==1))

    return sorted([matrix[tuple(ind)] for ind in nb_indices])


In [0]:
# This code will accept noise density as input from the user
noise_density = int(input('Please enter the noise density of the system in percentage: '))

Please enter the noise density of the system in percentage: 10


In [0]:
class C_O_F(object):
	"""This is the class that consist all utility functions that helps in calculating C_O_F"""
	def __init__(self, image):
		super(C_O_F, self).__init__()

		self.image = image
		self.gaussian_noise = self.noisy('gaussian', self.image)
		self.salt_and_pepper_noise = self.noisy('s&p', self.image)


	def getImage(self):
		"""
		Get the Pixel Ratio of the given image... i.e (256 x 256 pixels)
		"""
		return self.image.shape

	def getImageSize(self):
		"""
		Get the image size, which is number of pixels in the given image... 65536 pixels
		"""
		return self.image.size

	def noisy(self, noise_typ, image):
		"""
			Noise: This function will apply the impulse noise to the given image.
			Parameter:
        ----------
            noise_typ : Type of Noise to apply...i.e gaussian, salt-and-pepper.
            imaege : The coordinates of the given Matrix
		"""
		density = noise_density/100

		if noise_typ == "gaussian":
			row,col = image.shape
			mean = 0
					#var = 0.1
					#sigma = var**0.5
			gauss = np.random.normal(mean, density, (row,col))
			# gauss = gauss.reshape(row,col)
			noisy = image + gauss
			return noisy
		elif noise_typ == "s&p":
			row,col = image.shape
			s_vs_p = density
			amount = 0.004
			out = image
					# Salt mode
			num_salt = np.ceil(amount * image.size * s_vs_p)
			coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
			out[coords] = 1

					# Pepper mode
			num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
			coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
			out[coords] = 0
			return out
		else:
			raise ValueError("Noise unknown; you can only use either 'gauss' or 's&p', '{}' given".format(noise_typ))
	 
	def noise_and_nonNoise(self, original_image, noisy_image):
		"""
			noise_and_nonNoise: This function will 
			Parameter:
        ----------
            noise_typ : Type of Noise to apply...i.e gaussian, salt-and-pepper.
            imaege : The coordinates of the given Matrix
		"""
		rows, cols = original_image.shape
		original_pixels = []
		noisy_pixels = []

		# map = []
		for i in range(rows):
			for j in range(cols):
				if float(original_image[i][j]) == np.round(noisy_image[i][i]):
					original_pixels.append(0)
				else:
					noisy_pixels.append(1)
		 			# map[i][j] = 0

		return original_pixels, noisy_pixels

	def generateBinaryMap(self):
		"""
			generateBinaryMap: This function will generate binary map of the given noisy image 
													(0 for original pixels, 1 for noisy pixels)
		"""
		original_image = self.image
		noisy_image = self.gaussian_noise
		rows, cols = original_image.shape

		return [[0 if float(original_image[i][j]) == np.round(noisy_image[i][i]) else 1 for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Generating Binary Map...")]

	def comparePixels(self):
		"""
		comparePixels: This function will compare the pixels between two images
		"""
		gauss_noise = self.noise_and_nonNoise(self.image, self.gaussian_noise)
		sp_noise = self.noise_and_nonNoise(self.image, self.salt_and_pepper_noise)

		gauss_original_pixels, gauss_noisy_pixels = gauss_noise
		sp_original_pixels, sp_noisy_pixels = sp_noise

		return (len(gauss_original_pixels), len(gauss_noisy_pixels)), (len(sp_original_pixels), len(sp_noisy_pixels))

	def getNeighbors(self):
		"""
			This function return the dictionary of pixels
			KEYS: is the individual center pixel.
			VALUES: is the list of Neighbors of the key pixel.
		"""
		rows, cols = self.getImage()

		pixel_dict = dict()

		for i in range(rows):
			for j in range(cols):
				pixel_dict[self.image[i][j]] = NearestNeighbors(self.image, [i, j])

		return pixel_dict

	# Set based nearest (SBN) trail.
	# {(p1, p2), (p2,p3), (p3, p4), (p4, p5), (p5, p6), (p6, p7), (p7, p8)(p8, p9)}
	def sBNTrail(self):
		"""
		sBNTrail: this function will return the SBN trail
		"""
		pixels = self.getNeighbors()
		
		return pixels

	def sBNTrailCost(self, mat, i, j):
		"""
		sBNTrailCost: This function will calculate the SBN trail cost
		"""
		p = []
		p = NearestNeighbors(mat, [i, j])
		p.append(mat[i][j])
		p = sorted(p)
		l = len(p)-1
		e = []
		for i in range(l):
			e.append((p[i+1] - p[i]))
		return sorted(e)

	def cof(self):
		"""
		This function will calculate the COF
		"""
		distances_list = np.array(self.get_all_ac_dist())

		rows = distances_list.shape[0]
		cols = distances_list.shape[1]

		return [[(distances_list[i][j]/(np.sum(NearestNeighbors(distances_list, [i, j]))/len(NearestNeighbors(distances_list, [i, j])))) for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Calculating COF...")]

	def get_all_ac_dist(self):
		"""
		This fuction will calculate the average chaining distance of all the pixels in the given images.
		"""
		image = self.gaussian_noise
		a, b = self.getImage()
				
		return [[self.get_ac_dist(image, i, j) for i in range(a)] for j in range(b)]

	def get_ac_dist(self, img, i, j):
		"""
		get_ac_dist: This fuction will calculate the average chaining distance of the given pixel in a given coordinates.

		"""
		acd = []
		sbn_path_trail_cost = self.sBNTrailCost(img, i, j)
		k = len(NearestNeighbors(img, [i, j]))
		
		for i, sbn_cost in enumerate(sbn_path_trail_cost):
			acd.append(((2 * (k - (i+1)))/(k * (k - 1))) * sbn_cost)
		ac_dist = (np.sum(acd))

		return ac_dist

	def ratioOfPixels(self):
		"""
		ratioOfPixels: This function will return the ratio of noisy pixels and original pixels
		"""
		pixels = self.comparePixels()

		gauss, sp = pixels

		gauss_original_ratio = ((gauss[0]/(gauss[0]+gauss[1])) * 100)
		gauss_noisy_ratio = ((gauss[1]/(gauss[0]+gauss[1])) * 100)
	
		sp_original_ratio = ((sp[0]/(sp[0]+sp[1])) * 100)
		sp_noisy_ratio = ((sp[1]/(sp[0]+sp[1])) * 100)

		return (gauss_original_ratio, gauss_noisy_ratio), (sp_original_ratio, sp_noisy_ratio)
	
	def cof_value_to_binary(self, value):
		"""
		This function will convert the calculated COF value to Binary Value...i.e 0 = original, 1 = noisy
		"""
		value = np.array(value)
		rows, cols = value.shape
		
		cof_v = [[int(round(value[i][j])) for i in range(rows)] for j in range(cols)]

		cof_binary = [[1 if cof_v[i][j] >= 1 else 0 for i in range(rows)] for j in range(cols)]

		return cof_binary
	
	def detects(self):
		"""
		Produce final result
		"""
		cof_ = self.cof()
		cof_value = np.array(self.cof_value_to_binary(cof_))
		binary_value = np.array(self.generateBinaryMap())
		rows, cols = binary_value.shape

		return [['TP' if binary_value[i][j] == 1 and cof_value[i][j] == 1 else 'FP' if binary_value[i][j] == 1 and cof_value[i][j] == 0 else 'FN' if binary_value[i][j] == 0 and cof_value[i][j] == 1 else 'TN' if binary_value[i][j] == 0 and cof_value[i][j] == 0 else 11 for i in range(rows)] for j in tqdm_notebook(range(cols), desc="Detecting noisy pixels...")]



In [0]:
img = Image.open('lena.tif')


image = np.array(img, dtype=float)

cof_object = C_O_F(image)
tp = []
tn = []
fp = []
fn = []
pixels = cof_object.detects()
pixels = np.array(pixels)
rows, cols = pixels.shape
for i in range(rows):
	for j in range(cols):
		if pixels[i][j] == 'TP':
			tp.append(pixels[i][j])
		elif pixels[i][j] == 'TN':
			tn.append(pixels[i][j])
		elif pixels[i][j] == 'FP':
			fp.append(pixels[i][j])
		elif pixels[i][j] == 'FN':
			fn.append(pixels[i][j])
		else:
			print(None)
	 
print('\n\n')
print("True Positive = {}".format(len(tp)))
print("True Negative = {}".format(len(tn)))
print("False Positive = {}".format(len(fp)))
print("False Negative = {}".format(len(fn)))





True Positive = 60853
True Negative = 42
False Positive = 3978
False Negative = 663
